In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
ratings = pd.read_csv("../Data/ratings.csv")
print(ratings.head())

   userId  movieId  rating   timestamp
0     877     4155       5  1651201566
1     305     7661       2  1639553712
2     381     8423       2  1610704432
3     208     6433       1  1650223767
4      47     7752       4  1663998365


In [3]:
movies = pd.read_csv("../Data/movies.csv")
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [4]:
n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")

Number of ratings: 100836
Number of unique movieId's: 9742
Number of unique users: 999
Average ratings per user: 100.94
Average ratings per movie: 10.35


User Rating Frequency

In [5]:
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
print(user_freq.head())

   userId  n_ratings
0       1        120
1       2        105
2       3         89
3       4        100
4       5        107


Movie Rating Analysis

In [6]:
mean_rating = ratings.groupby('movieId')[['rating']].mean()
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]
ratings[ratings['movieId']==highest_rated]
ratings[ratings['movieId']==lowest_rated]

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

## User-Item Matrix Creation

This block creates a sparse user-item matrix using csr_matrix from scipy. It also generates mappings between user and movie IDs and their corresponding indices for use in the matrix.

- csr_matrix: creates a sparse matrix (Compressed Sparse Row) from the user-item ratings data to save memory.
- user_mapper and movie_mapper: create dictionaries that map user IDs and movie IDs to indices in the sparse matrix.
- user_inv_mapper and movie_inv_mapper: create reverse dictionaries that map matrix indices back to user IDs and movie IDs.
- user_index and movie_index: create lists of indices for users and movies from the ratings DataFrame.
- X = csr_matrix(...): constructs the sparse user-item matrix X with the given user and movie indices and ratings.

In [7]:
from scipy.sparse import csr_matrix

def create_matrix(df):
    
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
 
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
 
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
    
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

In [8]:
X

<9742x999 sparse matrix of type '<class 'numpy.int64'>'
	with 100333 stored elements in Compressed Sparse Row format>

Movie Similarity Analysis

In [9]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []
    
    if movie_id not in movie_mapper:
        print(f"Movie ID {movie_id} not found in movie_mapper!")
        return []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k += 1  
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    
    for i in range(0, k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    
    neighbour_ids.pop(0) 
    return neighbour_ids

Movie Recommendation with respect to Users Preference

In [12]:
def recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10):
    df1 = ratings[ratings['userId'] == user_id]

    movie_id = df1[df1['rating'] == max(df1['rating'])]['movieId'].iloc[0]

    movie_titles = dict(zip(movies['movieId'], movies['title']))

    movie_title = movie_titles[movie_id]

    similar_ids = find_similar_movies(movie_id, X, k)

    print(f"Since you watched {movie_title}, you might also like:")
    
    for i in similar_ids:
        if i in movie_titles:
            print(movie_titles[i])

user_id = 150 
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10)

    

Since you watched Miller's Crossing (1990), you might also like:
Flawless (1999)
Lilya 4-Ever (Lilja 4-ever) (2002)
Bells of St. Mary's, The (1945)
Dark City (1998)
Cradle 2 the Grave (2003)
Japanese Story (2003)


In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []
    
    if movie_id not in movie_mapper:
        print(f"Movie ID {movie_id} not found in movie_mapper!")
        return []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k += 1  
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    
    for i in range(0, k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    
    neighbour_ids.pop(0) 
    return neighbour_ids

def recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10):
    df1 = ratings[ratings['userId'] == user_id]

    movie_id = df1[df1['rating'] == max(df1['rating'])]['movieId'].iloc[0]

    movie_titles = dict(zip(movies['movieId'], movies['title']))

    movie_title = movie_titles[movie_id]

    similar_ids = find_similar_movies(movie_id, X, k)

    print(f"Since you watched {movie_title}, you might also like:")
    
    for i in similar_ids:
        if i in movie_titles:
            print(movie_titles[i])

user_id = 150 
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10)